## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Determine the number of unique values in each column.
unique_value_counts = application_df.nunique()
print(unique_value_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [5]:
# Choose a cutoff value for binning
cutoff_value = 500  # You can adjust this value as needed

# Create a list of application types to be replaced with "Other"
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful
print(application_df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_gt1 = classification_counts[classification_counts > 1]
print(classification_counts_gt1)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [8]:
# Choose a cutoff value for binning
cutoff_value = 1883  # You can adjust this value as needed

# Create a list of classifications to be replaced with "Other"
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure binning was successful
print(application_df['CLASSIFICATION'].value_counts())

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop(columns=['IS_SUCCESSFUL'])
y = application_df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

### OPTIMIZATION ATTEMPT #1 - Addition of more neural units and layer units

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train_scaled.shape[1]))


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=60, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation='relu'))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 60)                4860      
                                                                 
 dense_2 (Dense)             (None, 40)                2440      
                                                                 
 dense_3 (Dense)             (None, 1)                 41        
                                                                 
Total params: 10,861
Trainable params: 10,861
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Train the model with Early Stopping
nn.fit(X_train_scaled, y_train, epochs=100, verbose=2)

Epoch 1/100
804/804 - 2s - loss: 0.5718 - accuracy: 0.7194 - 2s/epoch - 3ms/step
Epoch 2/100
804/804 - 1s - loss: 0.5574 - accuracy: 0.7279 - 1s/epoch - 1ms/step
Epoch 3/100
804/804 - 1s - loss: 0.5537 - accuracy: 0.7283 - 1s/epoch - 1ms/step
Epoch 4/100
804/804 - 1s - loss: 0.5527 - accuracy: 0.7293 - 1s/epoch - 1ms/step
Epoch 5/100
804/804 - 1s - loss: 0.5510 - accuracy: 0.7309 - 994ms/epoch - 1ms/step
Epoch 6/100
804/804 - 1s - loss: 0.5501 - accuracy: 0.7310 - 1s/epoch - 1ms/step
Epoch 7/100
804/804 - 1s - loss: 0.5490 - accuracy: 0.7320 - 982ms/epoch - 1ms/step
Epoch 8/100
804/804 - 1s - loss: 0.5483 - accuracy: 0.7324 - 1s/epoch - 1ms/step
Epoch 9/100
804/804 - 1s - loss: 0.5475 - accuracy: 0.7330 - 1s/epoch - 1ms/step
Epoch 10/100
804/804 - 1s - loss: 0.5469 - accuracy: 0.7331 - 963ms/epoch - 1ms/step
Epoch 11/100
804/804 - 1s - loss: 0.5459 - accuracy: 0.7347 - 985ms/epoch - 1ms/step
Epoch 12/100
804/804 - 1s - loss: 0.5462 - accuracy: 0.7335 - 1s/epoch - 1ms/step
Epoch 13/100


In [15]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5674 - accuracy: 0.7284 - 449ms/epoch - 2ms/step
Loss: 0.5674264430999756, Accuracy: 0.728396475315094


In [16]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimisation_1.h5")

### OPTIMIZATION ATTEMPT #2 - Revision of activation options and addition of more epochs

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train_scaled.shape[1]))


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=60, activation='tanh'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation='tanh'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation='relu'))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 80)                3520      
                                                                 
 dense_5 (Dense)             (None, 60)                4860      
                                                                 
 dense_6 (Dense)             (None, 40)                2440      
                                                                 
 dense_7 (Dense)             (None, 20)                820       
                                                                 
 dense_8 (Dense)             (None, 1)                 21        
                                                                 
Total params: 11,661
Trainable params: 11,661
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 2s 1ms/step - loss: 0.5698 - accuracy: 0.7204
Epoch 2/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5556 - accuracy: 0.7281
Epoch 3/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5534 - accuracy: 0.7301
Epoch 4/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5525 - accuracy: 0.7302
Epoch 5/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5506 - accuracy: 0.7318
Epoch 6/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5501 - accuracy: 0.7315
Epoch 7/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5494 - accuracy: 0.7325
Epoch 8/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5490 - accuracy: 0.7323
Epoch 9/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7331
Epoch 10/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5478 - accura

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5713 - accuracy: 0.7286 - 407ms/epoch - 2ms/step
Loss: 0.5713024735450745, Accuracy: 0.7286297082901001


In [21]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimisation_2.h5")

### OPTIMIZATION ATTEMPT #3 - Revision of input variables and PCA

In [22]:
application_df_encoded.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [23]:
# Drop 'STATUS' and 'SPECIAL_CONSIDERATIONS' columns
application_df_encoded = application_df_encoded.drop(columns=['STATUS', 'SPECIAL_CONSIDERATIONS_N', 'SPECIAL_CONSIDERATIONS_Y'])
application_df_encoded.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,False,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,108590,1,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,5000,0,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
3,6692,1,False,False,False,True,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
4,142590,1,False,False,False,True,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False


In [24]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop(columns=['IS_SUCCESSFUL'])
y = application_df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2, test_size=0.20)

In [25]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [26]:
# Determine the number of Principal Components (PCA) to retain
from sklearn.decomposition import PCA

# Choose the desired explained variance ratio (e.g., 95%)
desired_explained_variance_ratio = 0.90

# Initialize PCA
pca = PCA()

# Fit PCA to the data
pca.fit(X_train_scaled)

PCA()

In [27]:
# Calculate cumulative explained variance ratio
explained_variance_ratio_cumulative = pca.explained_variance_ratio_.cumsum()

# Determine the number of components to retain
num_components_to_retain = len(explained_variance_ratio_cumulative[explained_variance_ratio_cumulative < desired_explained_variance_ratio]) + 1

# Print the number of components to retain
print(f'Number of components to retain: {num_components_to_retain}')

Number of components to retain: 27


In [28]:

# Create and fit a new PCA instance with the determined number of components
pca = PCA(n_components=num_components_to_retain)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)


In [29]:
X_train_pca.shape

(27439, 27)

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation='relu', input_dim=X_train_pca.shape[1]))


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=60, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 100)               2800      
                                                                 
 dense_10 (Dense)            (None, 80)                8080      
                                                                 
 dense_11 (Dense)            (None, 60)                4860      
                                                                 
 dense_12 (Dense)            (None, 1)                 61        
                                                                 
Total params: 15,801
Trainable params: 15,801
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
# Train the model
nn.fit(X_train_pca, y_train, epochs=150)

Epoch 1/150
858/858 [==============================] - 2s 1ms/step - loss: 0.5708 - accuracy: 0.7196
Epoch 2/150
858/858 [==============================] - 1s 1ms/step - loss: 0.5571 - accuracy: 0.7274
Epoch 3/150
858/858 [==============================] - 1s 1ms/step - loss: 0.5537 - accuracy: 0.7303
Epoch 4/150
858/858 [==============================] - 1s 1ms/step - loss: 0.5518 - accuracy: 0.7293
Epoch 5/150
858/858 [==============================] - 1s 1ms/step - loss: 0.5512 - accuracy: 0.7296
Epoch 6/150
858/858 [==============================] - 1s 1ms/step - loss: 0.5508 - accuracy: 0.7315
Epoch 7/150
858/858 [==============================] - 1s 1ms/step - loss: 0.5490 - accuracy: 0.7321
Epoch 8/150
858/858 [==============================] - 1s 1ms/step - loss: 0.5488 - accuracy: 0.7329
Epoch 9/150
858/858 [==============================] - 1s 2ms/step - loss: 0.5486 - accuracy: 0.7340
Epoch 10/150
858/858 [==============================] - 1s 2ms/step - loss: 0.5480 - accura

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_pca, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5828 - accuracy: 0.7267 - 350ms/epoch - 2ms/step
Loss: 0.5827944874763489, Accuracy: 0.7266764044761658


In [34]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimisation_3.h5")